In [1]:
! python3 --version

Python 3.6.8


In [2]:
from pre_process import Doc2vec_wrapper

In [3]:
doc2vec_model = Doc2vec_wrapper(json_path='/mnt/ceph/cme/30Ketds/30Kmetadata.json', n_docs=2)

Initializing preprocessor object. Downloading NLTK stopwords...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
doc2vec_model.generate_tokens()

In [5]:
doc2vec_model.load_model_and_build_vocab(epochs=1)

In [6]:
doc2vec_model.train()

In [3]:
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import os
import string
import numpy as np

[nltk_data] Downloading package punkt to /home/pkmandke/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pkmandke/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
gensim.__version__, nltk.__version__, pd.__version__, np.__version__

('3.8.1', '3.4.5', '0.25.2', '1.17.3')

In [ ]:
N = 10000
#ETD_PATH = '/mnt/ceph/cme/thesis/'
ETD_PATH = '../../thesis/'
list_of_docs = os.listdir(ETD_PATH)[:2]
len(list_of_docs)

In [ ]:
def data_generator():
    
    for idx, folder in enumerate(list_of_docs):
        for _ in os.listdir(ETD_PATH + folder):
            file_name = ''
            if _.endswith('.pdf.txt') or _.endswith('.PDF.txt'):
                file_name = _
                break
        if file_name:
            with open(ETD_PATH + folder + '/' + file_name) as f:
                document = f.read()
                
                tokens = nltk.word_tokenize(document.lower().translate(str.maketrans('', '', string.punctuation)))
                stems = []
                
                for item in tokens:
                    if item in stopwords.words():
                        continue
                    stems.append(PorterStemmer().stem(item))

                #tokens = gensim.utils.simple_preprocess(document)
                
                yield gensim.models.doc2vec.TaggedDocument(stems, [folder])

In [ ]:
train_corpus = list(data_generator())
len(train_corpus), type(train_corpus[0])

In [ ]:
train_corpus[1][1]

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=20, min_count=2, epochs=5, workers=5)
model.build_vocab(train_corpus)

In [ ]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.docvecs[1]

In [ ]:
ranks = []
second_ranks = []
for doc_id, name in zip(range(2), ['10000', '10001']):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(name)
    ranks.append(rank)

    second_ranks.append(sims[0])

In [ ]:
second_ranks

In [ ]:
model.estimate_memory()

In [ ]:
model.save('../obj/doc2vec1')